In [25]:
import math
import pandas as pd

SUBS = {
    "0": "₀", "1": "₁", "2": "₂", "3": "₃", "4": "₄",
    "5": "₅", "6": "₆", "7": "₇", "8": "₈", "9": "₉"
}

def subscript(num):
    return "".join(SUBS[d] for d in str(num))

def varname(i):
    return f"x{subscript(i)}"

def truth_table(n, ones):
    return [1 if i in ones else 0 for i in range(2**n)]


def from_function_number(num):
    bits = list(map(int, bin(num)[2:]))
    N = len(bits)
    n = math.ceil(math.log2(N))

    if 2**n != N:
        bits = [0] * (2**n - N) + bits

    return bits, n

def bits_of(num, n):
    return [(num >> (n - 1 - i)) & 1 for i in range(n)]


def num_from_bits(bits):
    val = 0
    for b in bits:
        val = (val << 1) | (1 if b else 0)
    return val

def table(n, values):
    rows = []
    columns = [varname(i + 1) for i in range(n)] + ["f"]

    for i in range(2**n):
        bits = bits_of(i, n)
        f = ["1" if values[i] else "0"]
        rows.append([str(b) for b in bits] + f)

    df = pd.DataFrame(rows, columns=columns)

    return df.style.hide(axis="index")


def derivatives(n, values, i):
    rows = []
    other_vars = [j for j in range(n) if j != i]
    columns = [varname(i + 1) for i in other_vars] + ["f₀", "f₀ ⊕ f₁", "f₁"]
    weight = 0
    
    for a in range(2**max(0, n - 1)):
        other_bits = bits_of(a, n - 1) if n - 1 > 0 else []
        bits0 = other_bits.copy()
        bits0.insert(i, 0)
        
        bits1 = other_bits.copy()
        bits1.insert(i, 1)

        m0 = num_from_bits(bits0)
        m1 = num_from_bits(bits1)

        f0 = "1" if values[m0] else "0"
        f1 = "1" if values[m1] else "0"
        xor = "1" if f0 != f1 else "0"

        weight += f0 != f1
        rows.append([str(b) for b in other_bits] + [f0, xor, f1])

    df = pd.DataFrame(rows, columns=columns)

    return df.style.hide(axis="index"), weight
    

def menu():
    print("Выберите способ ввода функции:")
    print("1 - ввести единичные наборы (ones)")
    print("2 - ввести нулевые наборы (zeros)")
    print("3 - ввести номер функции (в десятичной форме)")
    choice = input("Ваш выбор: ")

    if choice == "1":
        ones = list(
            map(int, input("Введите номера единичных наборов через пробел: ").split())
        )
        n = max(ones).bit_length()
        values = truth_table(n, ones)
    elif choice == "2":
        zeros = list(
            map(int, input("Введите номера нулевых наборов через пробел: ").split())
        )
        n = max(zeros).bit_length()
        ones = [i for i in range(2**n) if i not in zeros]
        values = truth_table(n, ones)
    elif choice == "3":
        num = int(input("Введите номер функции (в десятичной форме): "))
        values, n = from_function_number(num)
    else:
        print("Неверный выбор")
        return
    
    print("\nТаблица истинности")
    display(table(n, values))
    
    for i in range(n):
        df, w = derivatives(n, values, i)

        print(f"\nПроизводная ∂f/∂{varname(i + 1)}")
        display(df)
        print(f"Вес проивзодной: {w}")

if __name__ == "__main__":
    menu()

Выберите способ ввода функции:
1 - ввести единичные наборы (ones)
2 - ввести нулевые наборы (zeros)
3 - ввести номер функции (в десятичной форме)

Таблица истинности


x₁,x₂,x₃,f
0,0,0,1
0,0,1,0
0,1,0,1
0,1,1,1
1,0,0,1
1,0,1,1
1,1,0,1
1,1,1,0



Производная ∂f/∂x₁


x₂,x₃,f₀,f₀ ⊕ f₁,f₁
0,0,1,0,1
0,1,0,1,1
1,0,1,0,1
1,1,1,1,0


Вес проивзодной: 2

Производная ∂f/∂x₂


x₁,x₃,f₀,f₀ ⊕ f₁,f₁
0,0,1,0,1
0,1,0,1,1
1,0,1,0,1
1,1,1,1,0


Вес проивзодной: 2

Производная ∂f/∂x₃


x₁,x₂,f₀,f₀ ⊕ f₁,f₁
0,0,1,1,0
0,1,1,0,1
1,0,1,0,1
1,1,1,1,0


Вес проивзодной: 2
